## The Code Explained
Let's break the code down.

In [ ]:
#include "ros/ros.h"
#pragma cling load("roscpp")

`ros/ros.h` is a convenience include that includes all the headers necessary to use the most common public pieces of the ROS system. 

In [ ]:
#include <sstream>
#include "std_msgs/String.h"

This includes the [std_msgs/String message](http://docs.ros.org/api/std_msgs/html/msg/String.html), which resides in the [std_msgs](http://wiki.ros.org/std_msgs) package. This is a header generated automatically from the `String.msg` file in that package. For more information on message definitions, see the [msg](http://wiki.ros.org/msg) page. 

In [ ]:
int argc = 1;
char arg0[] = "talker";
char* argv[] = {arg0, NULL};

In [ ]:
ros::init(argc, argv, "talker")

Initialize ROS. This allows ROS to do name remapping through the command line -- not important for now. This is also where we specify the name of our node. Node names must be unique in a running system.

The name used here must be a [base name](http://wiki.ros.org/Names#Graph), ie. it cannot have a `/` in it. 

In [ ]:
ros::NodeHandle n;

Create a handle to this process' node. The first `NodeHandle` created will actually do the initialization of the node, and the last one destructed will cleanup any resources the node was using. 

In [ ]:
ros::Publisher chatter_pub = n.advertise<std_msgs::String>("chatter", 10);

Tell the master that we are going to be publishing a message of type [std_msgs/String](http://docs.ros.org/api/std_msgs/html/msg/String.html) on the `topic` chatter. This lets the master tell any nodes listening on chatter that we are going to publish data on that topic. The second argument is the size of our publishing queue. In this case if we are publishing too quickly it will buffer up a maximum of 1000 messages before beginning to throw away old ones.

`NodeHandle::advertise()` returns a `ros::Publisher object`, which serves two purposes: 1) it contains a `publish()` method that lets you publish messages onto the topic it was created with, and 2) when it goes out of scope, it will automatically unadvertise. 

In [ ]:
ros::Rate loop_rate(10);

A `ros::Rate` object allows you to specify a frequency that you would like to loop at. It will keep track of how long it has been since the last call to `Rate::sleep()`, and sleep for the correct amount of time.

In this case we tell it we want to run at 10Hz. 

In [ ]:
int count = 0;
while (ros::ok())
{
    std_msgs::String msg;

    std::stringstream ss;
    ss << "hello world " << count;
    msg.data = ss.str();

    chatter_pub.publish(msg);

    ros::spinOnce();

    loop_rate.sleep();
    ++count;
}

By default `roscpp` will install a SIGINT handler which provides Ctrl-C handling which will cause `ros::ok()` to return false if that happens.

`ros::ok()` will return false if:
* a SIGINT is received (Ctrl-C)
* we have been kicked off the network by another node with the same name
* `ros::shutdown()` has been called by another part of the application.
* all `ros::NodeHandles` have been destroyed 

Once `ros::ok()` returns false, all ROS calls will fail. 

We broadcast a message on ROS using a message-adapted class, generally generated from a [msg file](http://wiki.ros.org/msg). More complicated datatypes are possible, but for now we're going to use the standard `String` message, which has one member: "data".

Now we actually broadcast the message to anyone who is connected. 

`ROS_INFO` and friends are our replacement for `printf/cout`. See the [rosconsole documentation](http://wiki.ros.org/rosconsole) for more information.

Calling `ros::spinOnce()` here is not necessary for this simple program, because we are not receiving any callbacks. However, if you were to add a subscription into this application, and did not have `ros::spinOnce()` here, your callbacks would never get called. So, add it for good measure. 

Now we use the `ros::Rate` object to sleep for the time remaining to let us hit our 10Hz publish rate.

Here's the condensed version of what's going on:
* Initialize the ROS system
* Advertise that we are going to be publishing [std_msgs/String](http://docs.ros.org/api/std_msgs/html/msg/String.html) messages on the `chatter` topic to the master
* Loop while publishing messages to chatter 10 times a second 